# Web Scraping using API
- When you type www.google.com in your browser's address bar, your computer is actually asking the www.google.com server for a web page; the server returns the page to your browser.
- APIs work much the same way, except instead of your web browser asking for a web page, your program asks for data. The API usually returns this data in JavaScript Object Notation (JSON) format.
  - GET request: to retrieve information from the OpenNotify API.
  -

### Crawling and Scraping
- Crawling refers to the process of automatically traversing through a website and following links to other pages in order to discover and index content.
- Scraping, on the other hand, refers to the process of extracting specific data elements from web pages. Scraping involves sending HTTP requests to web pages and then parsing the HTML or other markup language in the response to extract the desired data.
- Crawling focuses on discovering and indexing web pages, while scraping targets specific data extraction from those pages

# JSON format
- json: JavaScript Object Notation (자바 스크립트 객체 표기법)
- 데이터를 쉽게 '교환' 하고 '저장' 하기 위한 텍스트 기반의 데이터 교환 표준

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json
from bs4 import BeautifulSoup

In [2]:
# exercise 1 : a simple json format
obj = """
{
    "name": "Kim",
    "places_lived": ["Seoul", "Korea"],
    "pet": null,
    "siblings": [{"name": "Scott", "age":25, "pet":"Zuko"}]
}
"""

In [3]:
type(obj)

str

In [4]:
r = json.loads(obj)   # decoding (json --> dict)
print(r)
type(r)

{'name': 'Kim', 'places_lived': ['Seoul', 'Korea'], 'pet': None, 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'}]}


dict

In [5]:
json.dumps(r)    # encoding (dict --> json)

'{"name": "Kim", "places_lived": ["Seoul", "Korea"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}]}'

In [6]:
# Exercise 2 (from https://rfriend.tistory.com/474)
py_data = {

    "1.FirstName": "Gildong",
    "2.LastName": "Hong",
    "3.Age": 20,
    "4.University": "Hangook University",
    "5.Courses": [
        {
            "Classes": [
                "Probability",
                "Generalized Linear Model",
                "Categorical Data Analysis"
            ],
            "Major": "Statistics"
        },
        {
            "Classes": [
                "Data Structure",
                "Programming",
                "Algorithms"
            ],
            "Minor": "ComputerScience"
        }
    ]
}

In [7]:
type(py_data)

dict

In [8]:
py_data.keys()
# py_data.items()

dict_keys(['1.FirstName', '2.LastName', '3.Age', '4.University', '5.Courses'])

In [9]:
py_data.values()

dict_values(['Gildong', 'Hong', 20, 'Hangook University', [{'Classes': ['Probability', 'Generalized Linear Model', 'Categorical Data Analysis'], 'Major': 'Statistics'}, {'Classes': ['Data Structure', 'Programming', 'Algorithms'], 'Minor': 'ComputerScience'}]])

In [10]:
py_data['5.Courses']

[{'Classes': ['Probability',
   'Generalized Linear Model',
   'Categorical Data Analysis'],
  'Major': 'Statistics'},
 {'Classes': ['Data Structure', 'Programming', 'Algorithms'],
  'Minor': 'ComputerScience'}]

In [11]:
import json
json_str = json.dumps(py_data)
print(json_str)
type(json_str)

{"1.FirstName": "Gildong", "2.LastName": "Hong", "3.Age": 20, "4.University": "Hangook University", "5.Courses": [{"Classes": ["Probability", "Generalized Linear Model", "Categorical Data Analysis"], "Major": "Statistics"}, {"Classes": ["Data Structure", "Programming", "Algorithms"], "Minor": "ComputerScience"}]}


str

In [12]:
pd.Series(py_data)

1.FirstName                                               Gildong
2.LastName                                                   Hong
3.Age                                                          20
4.University                                   Hangook University
5.Courses       [{'Classes': ['Probability', 'Generalized Line...
dtype: object

In [13]:
pd.DataFrame(py_data)

,1.FirstName,2.LastName,3.Age,4.University,5.Courses
0,Gildong,Hong,20,Hangook University,"{'Classes': ['Probability', 'Generalized Linea..."
1,Gildong,Hong,20,Hangook University,"{'Classes': ['Data Structure', 'Programming', ..."


In [14]:
pd.DataFrame(py_data).iloc[:,-1]

0    {'Classes': ['Probability', 'Generalized Linea...
1    {'Classes': ['Data Structure', 'Programming', ...
Name: 5.Courses, dtype: object

In [15]:
pd.DataFrame.from_dict(py_data)

,1.FirstName,2.LastName,3.Age,4.University,5.Courses
0,Gildong,Hong,20,Hangook University,"{'Classes': ['Probability', 'Generalized Linea..."
1,Gildong,Hong,20,Hangook University,"{'Classes': ['Data Structure', 'Programming', ..."


In [16]:
pd.json_normalize(py_data)

,1.FirstName,2.LastName,3.Age,4.University,5.Courses
0,Gildong,Hong,20,Hangook University,"[{'Classes': ['Probability', 'Generalized Line..."


In [17]:
py_data['5.Courses']

[{'Classes': ['Probability',
   'Generalized Linear Model',
   'Categorical Data Analysis'],
  'Major': 'Statistics'},
 {'Classes': ['Data Structure', 'Programming', 'Algorithms'],
  'Minor': 'ComputerScience'}]

In [18]:
pd.json_normalize(py_data, "5.Courses")

,Classes,Major,Minor
0,"[Probability, Generalized Linear Model, Catego...",Statistics,NaN
1,"[Data Structure, Programming, Algorithms]",NaN,ComputerScience


In [19]:
pd.json_normalize(py_data, "5.Courses", ['3.Age'])

,Classes,Major,Minor,3.Age
0,"[Probability, Generalized Linear Model, Catego...",Statistics,NaN,20
1,"[Data Structure, Programming, Algorithms]",NaN,ComputerScience,20


# json_normalize (data, record_path, meta, ...)
- data: dict or list of dict
- record_path : decode 해줘야할 열 지정 [{}, {}, {} ....]
- meta : decode 하는 열과 동일 차원에 존재하는 열들 중 데이터 프레임에 포함시킬 열 선택

In [20]:
# JSON exercise3
# from https://pandas.pydata.org/pandas-docs/stable/reference/api/\
#              pandas.io.json.json_normalize.html
data = [{'state': 'Florida',
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [21]:
type(data), len(data)

(list, 2)

In [22]:
pd.json_normalize(data)

,state,shortname,counties,info.governor
0,Florida,FL,"[{'name': 'Dade', 'population': 12345}, {'name...",Rick Scott
1,Ohio,OH,"[{'name': 'Summit', 'population': 1234}, {'nam...",John Kasich


In [23]:
pd.json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [24]:
pd.json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


# YAML parsing
- YAML (short for "YAML Ain't Markup Language") is a human-readable data serialization format that is commonly used for configuration files, data exchange, and other structured data.
- pyyaml: Python library that enables parsing and serialization of data in YAML format (parsing and conversion)

In [25]:
%%writefile config.yaml
# sample YAML configuration file (generateed by ChatGPT)

server:
  host: localhost
  port: 8080

database:
  host: localhost
  port: 5432
  user: myuser
  password: mypassword

Writing config.yaml


In [26]:
!cat config.yaml

# sample YAML configuration file (generateed by ChatGPT)

server:
  host: localhost
  port: 8080

database:
  host: localhost
  port: 5432
  user: myuser
  password: mypassword


In [27]:
!pip install pyyaml

In [28]:
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    # config = yaml.load(f)        # for more complex types
print(config, type(config))
print(config['server']['host'])    # Output: localhost
print(config['database']['user'])  # Output: myuser

{'server': {'host': 'localhost', 'port': 8080}, 'database': {'host': 'localhost', 'port': 5432, 'user': 'myuser', 'password': 'mypassword'}} <class 'dict'>
localhost
myuser


# HTML Parsing
- before you do this example, try to see and run some example HTML files which are in this directory

In [29]:
from bs4 import BeautifulSoup

In [30]:
html_text = """
<html>
<body>
  <h1> reading web page with python </h1>
     <p> page analysis </p>
     <p> page alignment </p>
     <td>some text</td><td></td><td><p>more text</p></td><td>even <p>more text</p></td>
</body>
</html>
"""

In [31]:
soup = BeautifulSoup(html_text, 'html.parser')
soup


<html>
<body>
<h1> reading web page with python </h1>
<p> page analysis </p>
<p> page alignment </p>
<td>some text</td><td></td><td><p>more text</p></td><td>even <p>more text</p></td>
</body>
</html>

In [32]:
type(soup)

bs4.BeautifulSoup

In [33]:
soup.h1

<h1> reading web page with python </h1>

In [34]:
soup.h1.text.strip()

'reading web page with python'

In [35]:
soup.p

<p> page analysis </p>

In [36]:
soup.p.next_sibling.next_sibling

<p> page alignment </p>

In [37]:
soup.td.next_sibling.next_sibling

<td><p>more text</p></td>

In [38]:
print(soup.td.next_sibling, soup.td.next_sibling.text)

<td></td> 


In [39]:
html_text2 = """
<html>
<body>
  <h1 id="title"> reading web page with python </h1>
     <p id="body"> page analysis </p>
     <p> page alignment </p>
     <td>some text</td><td></td><td><p>more text</p></td><td>even <p>more text</p></td>
     <ul>
         <li><a href = "http://www.naver.com"> naver</a></li>
         <li><a href = "http://www.daum.net"> daum</a></li>
     </ul>
  <div id="xxx">
    <h1> Wiki-books store </h1>
    <ul class="item all">
      <li> introduction to game design </li>
      <li> introduction to python </li>
      <li> introduction to web design </li>
    </ul>
  </div>
</body>
</html>
"""

In [40]:
soup = BeautifulSoup(html_text2, 'html.parser')

In [41]:
soup


<html>
<body>
<h1 id="title"> reading web page with python </h1>
<p id="body"> page analysis </p>
<p> page alignment </p>
<td>some text</td><td></td><td><p>more text</p></td><td>even <p>more text</p></td>
<ul>
<li><a href="http://www.naver.com"> naver</a></li>
<li><a href="http://www.daum.net"> daum</a></li>
</ul>
<div id="xxx">
<h1> Wiki-books store </h1>
<ul class="item all">
<li> introduction to game design </li>
<li> introduction to python </li>
<li> introduction to web design </li>
</ul>
</div>
</body>
</html>

## access by tags

In [42]:
soup.find(id='title')

<h1 id="title"> reading web page with python </h1>

In [43]:
soup.find(id='body').text

' page analysis '

In [44]:
soup.find_all('p')

[<p id="body"> page analysis </p>,
 <p> page alignment </p>,
 <p>more text</p>,
 <p>more text</p>]

In [45]:
soup.find_all('li')

[<li><a href="http://www.naver.com"> naver</a></li>,
 <li><a href="http://www.daum.net"> daum</a></li>,
 <li> introduction to game design </li>,
 <li> introduction to python </li>,
 <li> introduction to web design </li>]

In [46]:
soup.find_all('li')[0]

<li><a href="http://www.naver.com"> naver</a></li>

In [47]:
soup.find_all('li')[0].text, soup.find_all('li')[0].attrs

(' naver', {})

In [48]:
soup.find_all('a')[0]

<a href="http://www.naver.com"> naver</a>

In [49]:
soup.find_all('a')[0].text, soup.find_all('a')[0].attrs

(' naver', {'href': 'http://www.naver.com'})

In [50]:
for aa in soup.find_all('a'):
    href = aa.attrs['href']
    text = aa.string
    print (text, "-->", href)

 naver --> http://www.naver.com
 daum --> http://www.daum.net


## access by regular expression

In [51]:
soup


<html>
<body>
<h1 id="title"> reading web page with python </h1>
<p id="body"> page analysis </p>
<p> page alignment </p>
<td>some text</td><td></td><td><p>more text</p></td><td>even <p>more text</p></td>
<ul>
<li><a href="http://www.naver.com"> naver</a></li>
<li><a href="http://www.daum.net"> daum</a></li>
</ul>
<div id="xxx">
<h1> Wiki-books store </h1>
<ul class="item all">
<li> introduction to game design </li>
<li> introduction to python </li>
<li> introduction to web design </li>
</ul>
</div>
</body>
</html>

In [52]:
import re
soup.find_all(re.compile("^p"))   # tags starting with a character 'p'

[<p id="body"> page analysis </p>,
 <p> page alignment </p>,
 <p>more text</p>,
 <p>more text</p>]

In [53]:
soup.find_all(re.compile("div" ))

[<div id="xxx">
 <h1> Wiki-books store </h1>
 <ul class="item all">
 <li> introduction to game design </li>
 <li> introduction to python </li>
 <li> introduction to web design </li>
 </ul>
 </div>]

In [54]:
soup.find_all(href=re.compile("^http://"))

[<a href="http://www.naver.com"> naver</a>,
 <a href="http://www.daum.net"> daum</a>]

## access by css (Cascading Style Sheets) selector

In [55]:
soup.select('h1')    # by tags

[<h1 id="title"> reading web page with python </h1>,
 <h1> Wiki-books store </h1>]

In [56]:
soup.select('#xxx')  # by id

[<div id="xxx">
 <h1> Wiki-books store </h1>
 <ul class="item all">
 <li> introduction to game design </li>
 <li> introduction to python </li>
 <li> introduction to web design </li>
 </ul>
 </div>]

In [57]:
soup.select('.item') # by class name

[<ul class="item all">
 <li> introduction to game design </li>
 <li> introduction to python </li>
 <li> introduction to web design </li>
 </ul>]

In [58]:
soup.select('div .item')  # multi-components(tag=div, class=item)

[<ul class="item all">
 <li> introduction to game design </li>
 <li> introduction to python </li>
 <li> introduction to web design </li>
 </ul>]

In [59]:
soup.select("#xxx > ul > li")  # hierarchy (child)

[<li> introduction to game design </li>,
 <li> introduction to python </li>,
 <li> introduction to web design </li>]

In [60]:
soup.select_one("#xxx > ul > li")  # hierarchy (child)

<li> introduction to game design </li>

In [61]:
soup.select("div li")   # hierarchy (div tag >>> ul tag) (descendants)

[<li> introduction to game design </li>,
 <li> introduction to python </li>,
 <li> introduction to web design </li>]

- find with classes

In [62]:
text = '<p class="body strikeout"></p>'

css_soup = BeautifulSoup(text, 'html.parser')
css_soup.find_all("p", class_="strikeout")  # can have multiple values for a class

[<p class="body strikeout"></p>]

In [63]:
css_soup.find_all("p", class_="body")

[<p class="body strikeout"></p>]

In [64]:
# If you want to search for tags that match two or more CSS classes,
# you should use a CSS selector:
css_soup.select("p.body.strikeout")

[<p class="body strikeout"></p>]

In [65]:
css_soup.select("p.strikeout")

[<p class="body strikeout"></p>]

# Example from JOBKOREA
- confirmed in 2023.7.13
- kr.indeed.com (search for 'data science', 'seoul')
- Remember that web pages are changing.

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# search for 'data scientist', 'seoul' in JobaKorea
url = 'https://www.jobkorea.co.kr/Search/?stext=data%20scientist&local=I000'
link = requests.get(url)
soup = BeautifulSoup(link.text, 'html.parser')

In [67]:
link.status_code

200

In [68]:
soup.select('.recruit-info')

[<div class="recruit-info" style="">
 <div class="list-filter-wrap">
 <p class="filter-text">총 <strong class="dev_tot">59</strong>건</p>
 <div class="btn-wrap">
 <button class="btn-list-filter pseudo-icn lazyBg blk-arw" type="button">정확도순</button>
 <div class="layer-comn layer-filter">
 <ul class="dev_list_filter" data-key="Ord">
 <li><button class="filter" data-val="ExactDesc" type="button">정확도순</button></li>
 <li><button class="filter" data-val="RegDtDesc" type="button">등록일순</button></li>
 <li><button class="filter" data-val="EditDtDesc" type="button">최신업데이트순</button></li>
 <li><button class="filter" data-val="ApplyCloseDtAsc" type="button">마감임박순</button></li>
 <li><button class="filter" data-val="ReadCntDesc" type="button">조회수순</button></li>
 <li><button class="filter" data-val="ApplicantDesc" type="button">지원자순</button></li>
 </ul>
 </div>
 </div>
 </div>
 <div class="lists">
 ﻿
 
 
 <div class="lists-cnt dev_list" total-count="59">
 <div class="list-default">
 <ul class="clear">
 <

In [69]:
len(soup.select('.recruit-info'))

1

In [70]:
ss = soup.select('.recruit-info')[0].select('.lists')
len(ss)

1

In [71]:
len(ss[0].select('.post'))

20

In [72]:
ss[0].select('.post')[10]

<div class="post">
<div class="post-list-corp">
<a class="name dev_view" href="/Recruit/GI_Read/42290495?Oem_Code=C1&amp;logpath=1&amp;stext=data scientist&amp;listno=11" nav-src="/Search/_ContentsGIRead?Gno=42290495&amp;Mem_Type_Code=C&amp;Mem_Sys_No=36694653" onclick="GA_Virtual_Dimension($(this).closest('.list-post[data-gno=42290495]').data('gainfo'));GA_Virtual('홈&gt;통합검색&gt;공고뷰', $(this).closest('.list-post[data-gno=42290495]').data('gavirturl'));GA_Event('통합검색_PC', '채용정보', '클릭');" title="링거스튜디오">링거스튜디오</a>
<button class="btn-fav pseudo-icn-old lazyBg heart dev_favor-36694653" onclick="GA_Event('통합검색_PC', '채용정보','관심_' + ($(this).hasClass('on')?'해제':'등록'));" type="button">관심기업</button>
</div>
<div class="post-list-info">
<a class="title dev_view" href="/Recruit/GI_Read/42290495?Oem_Code=C1&amp;logpath=1&amp;stext=data scientist&amp;listno=11" nav-src="/Search/_ContentsGIRead?Gno=42290495&amp;Mem_Type_Code=C&amp;Mem_Sys_No=36694653" onclick="GA_Virtual_Dimension($(this).closest('.li

In [73]:
ss[0].select('.post')[0].select('.post-list-info')[0].a.text.strip()

'[서울본사] Data Scientist 신입/경력 채용'

In [74]:
s0 = ss[0].select('.post')[0].select('.post-list-corp')
corp = s0[0].a.text
s0 = ss[0].select('.post')[0].select('.post-list-info')
title = s0[0].a.text.strip()
loc = s0[0].select('.loc.long')[0].text
date = s0[0].select('.date')[0].text
etc = s0[0].select('.etc')[0].text

In [75]:
corp, title, loc, date, etc

('드림씨아이에스',
 '[서울본사] Data Scientist 신입/경력 채용',
 '서울 종로구',
 '~07/23(일)',
 '빅데이터, 데이터분석')

In [76]:
ss = soup.select('.recruit-info')[0].select('.lists')
post = ss[0].select('.post')
print("There are a total of {} job postings.".format(len(post)))

corps, titles, locs, dates, etcs = [], [], [], [], []
for i in range(len(post)):
    s0 = post[i].select('.post-list-corp')
    corps.append(s0[0].a.text)
    s0 = post[i].select('.post-list-info')
    titles.append(s0[0].a.text.strip())
    locs.append(s0[0].select('.loc.long')[0].text)
    dates.append(s0[0].select('.date')[0].text)
    etcs.append(s0[0].select('.etc')[0].text)

There are a total of 20 job postings.


In [77]:
pd.DataFrame(np.c_[corps, titles, locs, dates, etcs],
             columns=["corps", "titles", "locs", "dates", "etcs"])

,corps,titles,locs,dates,etcs
0,드림씨아이에스,[서울본사] Data Scientist 신입/경력 채용,서울 종로구,~07/23(일),"빅데이터, 데이터분석"
1,엘지전자㈜,[HE본부] 마케팅 자동화 전문가(Data Scientist) 경력채용,서울 영등포구,~07/28(금),"빅데이터, 인공지능, 데이터마이닝, 클라우드아키텍트"
2,엘지전자㈜,[한국영업본부] Data Scientist 경력채용,서울 중구,~07/31(월),"데이터분석, 데이터파이프라인"
3,엘지전자㈜,[한국영업본부] Data Scientist/Data Engineer 경력채용,서울 중구,~07/31(월),"데이터분석, 데이터파이프라인"
4,케이티넥스알,각 부문별 경력사원 수시채용(Data Scientist),서울 송파구,상시채용,"빅데이터, 솔루션영업, 네트워크관리, data science, 데이터분석, yarn..."
5,㈜직방,[직방] Senior Data Scientist,서울 강남구 외,~08/03(목),"데이터시각화, BI, 모바일App, 데이터분석, 분석모델링"
6,비에이코리아㈜,Data Scientist 신입/경력자 채용,서울 영등포구,~08/31(목),"빅데이터, 딥러닝, 인공지능, 데이터분석"
7,㈜쇼퍼하우스,AI Data Scientist 채용 (경력),서울 강남구,~09/10(일),"B2B, 상품기획, B2C, 데이터시각화, 데이터분석, 분석모델링, 오픈마켓, 소셜커머스"
8,㈜델타엑스,AI R&D / 3D 비전 / 개발 / 보안 / Data Scientist 병역 특...,서울 중구,상시채용,"모바일앱개발, 소프트웨어개발, 솔루션, BigData, 딥러닝"
9,㈜에이브랩스,데이터 분석 (Data Scientist) 채용,서울 중구,상시채용,"BigData, 빅데이터, 데이터분석, 딥러닝, 인공지능"


# Exercise
- .text: extract the combined text content of an element and its descendants. (recommended)
- .string: extract the text content of an individual element without considering any nested elements.
- It's important to note that the string attribute can only be used when an element has a single string as its content. If an element has multiple strings or tags, string will return None.
- In summary, text returns the combined text content of an element and its descendants, while string returns the text content of an individual element without considering its children.


In [78]:
# .text and .string
html_text = """
<html>
<body>
  <td>some text</td>
  <td></td>
  <td><p>more text</p></td>
  <td>even <p>more text</p></td>"
</body>
</html>
"""
soup = BeautifulSoup(html_text, 'html.parser')

In [79]:
soup.find_all('td')

[<td>some text</td>,
 <td></td>,
 <td><p>more text</p></td>,
 <td>even <p>more text</p></td>]

In [80]:
for i in soup.find_all('td'):
    print(i.string)

some text
None
more text
None


In [81]:
for i in soup.find_all('td'):
    print(i.text)

some text

more text
even more text


### Request
- get
- put

In [82]:
import requests

# the following two are the same.
parameters = {"lat": 40.71, "lon": -74}
response = requests.get("http://api.open-notify.org/iss-now.json",
                        params=parameters)
print(response.content)

response = requests.get("http://api.open-notify.org/iss-now.json?lat=40.71&lon=-74")
print(response.content)

b'{"timestamp": 1689835502, "message": "success", "iss_position": {"latitude": "28.6522", "longitude": "-80.1510"}}'
b'{"timestamp": 1689835502, "message": "success", "iss_position": {"latitude": "28.6522", "longitude": "-80.1510"}}'


In [83]:
response.status_code

200

In [84]:
response = requests.get("http://api.open-notify.org/iss-now.json")

In [85]:
response.content

b'{"timestamp": 1689835502, "message": "success", "iss_position": {"latitude": "28.6522", "longitude": "-80.1510"}}'

- getting JSON from a request

In [86]:
# Make the same request we did

response = requests.get("http://api.open-notify.org/iss-now.json")

# Get the response data as a Python object.  Verify that it's a dictionary.
json_data = response.json()
print(type(json_data))
print(json_data)

<class 'dict'>
{'timestamp': 1689835502, 'message': 'success', 'iss_position': {'latitude': '28.6522', 'longitude': '-80.1510'}}


In [87]:
# Headers is a dictionary
print(response.headers)
content_type = response.headers['Content-Type']
content_type

{'Server': 'nginx/1.10.3', 'Date': 'Thu, 20 Jul 2023 06:45:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}


'application/json'